# 전처리 스크래치, 로컬 다커 및 모델 빌딩 파이프라인

In [88]:
import boto3
import sagemaker
import pandas as pd


region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"FraudScratchModelPackageGroupName"

## 데이터 세트 로딩 및 S3 업로드

In [89]:
data_dir = '../data'
df_train = pd.read_csv(f"{data_dir}/train.csv", index_col=0)
df_test = pd.read_csv(f"{data_dir}/test.csv", index_col=0)
df_dataset = pd.concat([df_train, df_test], axis=0)
df_dataset.reset_index().to_csv("{}/dataset.csv".format(data_dir), index=None)
print("df_train shape: ", df_train.shape)
print("df_test shape: ", df_test.shape)
print("df_dataset shape: ", df_dataset.shape)

df_train shape:  (16000, 45)
df_test shape:  (4000, 45)
df_dataset shape:  (20000, 45)


이제 데이터를 디폴트 버킷으로 업로드합니다. `input_data_uri` 변수를 통해 데이터셋의 위치를 저장하였습니다.

In [90]:
local_path = "../data/dataset.csv"
data_prefix = 'fraud2scratch'
base_uri = f"s3://{default_bucket}/{data_prefix}"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path, 
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/dataset.csv


## (3) 모델 빌딩 파이프라인에서 전처리 로직 실행

#### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



### 모델 빌딩 파이프라인 변수 생성



In [91]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)



input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


다음으로 프로세싱을 위한 `SKLearnProcessor`의 인스턴스를 생성합니다. 이 인스턴스는 `ProcessingStep`에서 사용합니다.

본 노트북에서 계속 사용할 `framework_version`값을 지정합니다.

`sklearn_processor` 인스턴스의 생성시 `processing_instance_type`과 `processing_instance_count` 파라미터가 사용된 것을 확인합니다.


### 전처리 스텝 단계 정의

In [92]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)
print("input_data: \n", input_data)

input_data: 
 s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/dataset.csv


In [93]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="FraudScratchProcess",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/output/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/output/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/output/test")
    ],
    code="fraud/preprocessing.py",
)

### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [107]:
from sagemaker.estimator import Estimator

model_prefix = 'fraud2scratch/model'
model_path = f"s3://{default_bucket}/{model_prefix}"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
#     instance_type=training_instance_type,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0
)

In [108]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
)

### 학습모델을 평가하기 위한 모델 평가단계 정의 

먼저 모델 평가용 프로세싱 단계에서 질행할 평가용 스크립트를 작성합니다. 

파이프라인 실행 후 `evaluation.json`파일을 통해 평가결과를 확인할 수 있습니다.

평가 스크립트는 `xgboost`를 사용하고 다음을 실행합니다.

* 모델을 로드합니다. 
* 테스트 데이터를 읽습니다. 
* 테스트 데이터에 대한 예측을 실행합니다. 
* 정확도, ROC곡선 등을 포함하는 분류보고서를 작성합니다. 
* 평가 디렉토리에 평가보고서를 저장합니다. 

In [109]:
%%writefile evaluation.py
import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import mean_squared_error


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    print("pickle version: ", pickle.format_version)    
    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)
    
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    
    X_test = xgboost.DMatrix(df.values)
    
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {
                "value": mse,
                "standard_deviation": std
            },
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting evaluation.py


In [110]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-abalone-eval",
    role=role,
)

In [111]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
step_eval = ProcessingStep(
    name="AbaloneEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="evaluation.py",
    property_files=[evaluation_report],
)

# 모델 빌딩 파이프라인 정의

In [112]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"FraudScratchPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        input_data,
    ],
#     steps=[step_process, step_train, step_eval],
    steps=[step_process, step_train],    
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [113]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [114]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraudscratchpipeline',
 'ResponseMetadata': {'RequestId': 'adc311de-9c17-42c9-a5a7-8d602057bbdc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'adc311de-9c17-42c9-a5a7-8d602057bbdc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Mon, 12 Apr 2021 06:42:02 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [115]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [116]:
# execution.describe()
# execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [119]:
execution.list_steps()

[{'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 4, 12, 6, 46, 5, 650000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 4, 12, 6, 49, 25, 220000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-9ct6szmb6rb0-fraudscratchtrain-zqjzac2pym'}}},
 {'StepName': 'FraudScratchProcess',
  'StartTime': datetime.datetime(2021, 4, 12, 6, 42, 4, 440000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 4, 12, 6, 46, 4, 696000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-9ct6szmb6rb0-fraudscratchprocess-mok5cnksmq'}}}]

### 참고: 프레세싱 잡 상세 보기

In [48]:
import boto3

client = boto3.client('sagemaker')
client.describe_processing_job(ProcessingJobName = 'pipelines-uspaw6qef2dr-fraudscratchprocess-q9lsxbslfn')

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/dataset.csv',
    'LocalPath': '/opt/ml/processing/input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-12-01-05-11-748/input/code/preprocessing.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'train',
    'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-12-01-05-02-817/output/train',
     'LocalPath': '/opt/ml/processing/output/train',
     'S3UploadM

### 참고: 프레세싱 잡 상세 보기

In [122]:
response = client.describe_training_job(TrainingJobName = 'pipelines-9ct6szmb6rb0-fraudscratchtrain-zqjzac2pym')
response['ModelArtifacts']['S3ModelArtifacts']

's3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/model/pipelines-9ct6szmb6rb0-FraudScratchTrain-zQJzAc2pYM/output/model.tar.gz'